In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
import os
import math
import read_nd2
from cell import Cell

In [ ]:
# set working directory
print(os.getcwd())
folder_path = "../src"

if os.path.exists(folder_path) and os.path.isdir(folder_path):
    # Change the current working directory to the specified folder
    os.chdir(folder_path)
    print(f"Current working directory set to: {folder_path}")
else:
    print(f"The folder '{folder_path}' does not exist.")

In [ ]:
# function to check if pixel is within a contour - is flourescence within a cell

def is_pixel_inside_contour(pixel, contour):
    """
    Checks if a pixel is inside a contour.

    Args:
        pixel: A tuple (x, y) representing the pixel's coordinates.
        contour: A list of contour points, obtained from cv2.findContours.

    Returns:
        True if the pixel is inside the contour, False otherwise.
    """
    x, y = pixel
    point = (x, y)
    return cv2.pointPolygonTest(contour, point, False) >= 0



In [ ]:
# # downscale image
# scale_percent = 60 # percent of original size
# width = int(image.shape[1] * scale_percent / 100)
# height = int(image.shape[0] * scale_percent / 100)
# dim = (width, height)

# #

In [ ]:
# function to get the center of a contour
def get_center(contour):
    # Calculate moments for the contour
    M = cv2.moments(contour)

    # Calculate the centroid (center) of the contour
    if M["m00"] != 0:
        cx = int(M["m10"] / M["m00"])
        cy = int(M["m01"] / M["m00"])
    else:
        cx, cy = 0, 0  # Handle division by zero if the contour has no area

    # add centroid to list of cell centers
    return((cx,cy))

In [ ]:
# read the image
#image = cv2.imread('pngs/cells_C0_T178.png')

# function to segment an image with watershedding
# input: image to segment
# output: list of cells (1 for each cell)
def do_watershed(img):
    img = cv2.normalize(img, None, 0, 255, cv2.NORM_MINMAX, dtype=cv2.CV_8U)
    # convert to grayscale
    grayscale = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    # threshold image 
    thresh = cv2.threshold(grayscale,0,255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)[1]
    
    # get sure background area
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3))
    sure_bg = cv2.dilate(thresh, kernel, iterations=3) 
    
    # Distance transform
    dist = cv2.distanceTransform(thresh, cv2.DIST_L2, 5)

    # get foreground area
    ret, sure_fg = cv2.threshold(dist, 0.2 * dist.max(), 255, cv2.THRESH_BINARY)
    sure_fg = sure_fg.astype(np.uint8) 
    
    # get unknown area
    unknown = cv2.subtract(sure_bg, sure_fg)
    
    # connected components
    ret, markers = cv2.connectedComponents(sure_fg)
    markers = markers.astype(np.int32)
    # Add one to all labels so that background is not 0, but 1
    markers += 1
    # mark the region of unknown with zero
    markers[unknown == 255] = 0
    
    # apply watershed algorithm
    markers = cv2.watershed(img, markers)
    
    labels = np.unique(markers)

    cells = []
    for label in labels[2:]:
        # Create a binary image in which only the area of the label is in the foreground 
        # and the rest of the image is in the background   
        target = np.where(markers == label, 255, 0).astype(np.uint8)

        # Perform contour extraction on the created binary image
        contours, hierarchy = cv2.findContours(
            target, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE
        )
        center = get_center(contours[0])
        curr_cell = Cell()
        curr_cell.add_coordinate(center)
        curr_cell.add_contour(contours[0])
        cells.append(curr_cell)
    
    return cells
    

In [ ]:
# function to loop over directory and read in all images
def read_images_from_folder(folder_path):
    image_list = []

    # Check if the folder exists
    if not os.path.exists(folder_path):
        print(f"The folder '{folder_path}' does not exist.")
        return []

    # Get a list of files in the folder
    file_list = os.listdir(folder_path)

    # Iterate through the files and read images
    for file_name in file_list:
        file_path = os.path.join(folder_path, file_name)

        # Check if the file is an image (you can add more image extensions if needed)
        if file_name.lower().endswith(('.png', '.jpg', '.jpeg', '.gif', '.bmp', '.tif')):
            image = cv2.imread(file_path)
            
            # Append the image to the list if it was successfully read
            if image is not None:
                image_list.append(image)
    
    return image_list
    
def dist_between_points(coord_a, coord_b):
    x1, y1 = coord_a
    x2, y2 = coord_b
    
    # Calculate the Euclidean distance between the two points
    distance = math.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2)
    return distance


In [ ]:
def link_cell(parent_cell, curr_frame_cells):
    # get most recent coordinate of cell from dictionary
        prev_point = parent_cell.get_most_recent_coord()
        # initialize list of distances
        dists = {}
        for curr_cell in curr_frame_cells:
            curr_point = curr_cell.coords[0]
            # get distance from every other point
            dists[curr_cell] = dist_between_points(prev_point, curr_point)

        # add coord with smallest distance 
        curr_smallest_dists = [float('inf'), float('inf')]
        closest_cells = [None, None]
        for dist_cell in dists:
            if dists[dist_cell] < curr_smallest_dists[0]:
                curr_smallest_dists[0] = dists[dist_cell]
                closest_cells[0] = (dist_cell, curr_smallest_dists[0]) 
            elif dists[dist_cell] < curr_smallest_dists[1]:
                curr_smallest_dists[1] = dists[dist_cell]
                closest_cells[1] = (dist_cell, curr_smallest_dists[1])
        
        # return two closest cells (most likely and potential child) and their distances
        return closest_cells

In [ ]:
# function to resolve conflicts in cell tracking
def resolve_conflicts(candidates):
    # loop over candidates
    resolved_tracks = []
    for i in range(0, len(candidates)):
        candidate = candidates[i]

        most_likely_cell = candidate[0][0]
        most_likely_dist = candidate[0][1]
        pot_child_cell = candidate[1][0]
        pot_child_dist = candidate[1][1]

        # check if potential child is most likely of any other cell
        # if not, check if it's a potential child of any others
        for comparison_candidate in candidates:
            comp_most_likely_cell = comparison_candidate[0][0]
            comp_most_likely_dist = comparison_candidate[0][1]
            comp_pot_child_cell = comparison_candidate[1][0]
            comp_pot_child_dist = comparison_candidate[1][1]

            if pot_child_cell == comp_most_likely_cell:
                resolved_tracks.append([most_likely_cell])
                break
            elif pot_child_cell == comp_pot_child_cell:
                # check distances between child and two master cells
                if pot_child_dist <= comp_pot_child_dist:
                    # if child is closer to current cell, add it to resolved track
                    resolved_tracks.append([most_likely_cell, pot_child_cell])
                else:
                    resolved_tracks.append([most_likely_cell])
                    break

    return resolved_tracks

In [ ]:
def link_next_frame(master_cell_list, curr_frame):
    # get all the cells in the current frame
    curr_frame_cells = do_watershed(curr_frame)
    # check cells against previous frame
    candidates = []
    for i in range(0, len(master_cell_list)):
        # get closest two cells 
        cell = master_cell_list[i]
        closest_cells = link_cell(cell, curr_frame_cells)
        candidates.append(closest_cells)

    # now we have list of possible candidates
    # want to make sure that each cell has one & only one candidate child (or 2 in the case it divided)
    resolved_tracks = resolve_conflicts(candidates)

    # loop over resolved tracks and add to master cell list
    for i in range(0, len(resolved_tracks)):
        # if there's one child, add it to master cell object
        if len(resolved_tracks[i]) == 1:
            # add coord and contour from closest cell to master cell object
            master_cell_list[i].add_coordinate(resolved_tracks[i][0].coords[0])
            master_cell_list[i].add_contour(resolved_tracks[i][0].contours[0])
        elif len(resolved_tracks[i]) == 2:
            # create new cell in master list with parent history
            new_cell = Cell(coords = master_cell_list[i].coords,
                                 contours = master_cell_list[i].contours,
                                 parent = master_cell_list[i])
            # give child its tracking
            new_cell.add_coordinate(resolved_tracks[i][1].coords[0])
            new_cell.add_contour(resolved_tracks[i][1].contours[0])
            master_cell_list.append(new_cell)
            
            # give parent its child
            master_cell_list[i].add_child(new_cell)
            # give parent its tracking info
            master_cell_list[i].add_coordinate(resolved_tracks[i][0].coords[0])
            master_cell_list[i].add_contour(resolved_tracks[i][0].contours[0])

    return(master_cell_list)


        

In [ ]:
# read in all frames from folder
# detect all cells in first frame
# for each subsequent frame, do track linking with previous frame



# read in all images
movie = read_nd2.read_nd2("../doc/WellD01_ChannelmIFP,mCherry,YFP_Seq0000.nd2")
site0 = read_nd2.get_site_data(movie, 0)
frame0 = read_nd2.get_frame_data(movie, 0, 0)


# create master cell list from first frame
master_cells = do_watershed(frame0)
# loop over remaining frames and do track linking
for i in range(1, len(site0)):
    curr_frame = site0[i]
    master_cells = link_next_frame(master_cells, curr_frame)


In [ ]:
for cell in master_cells:
    print(cell)